In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_validate,KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

In [37]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [38]:
train_values = pd.read_csv('train_values.csv')
train_labels = pd.read_csv('train_labels.csv')
test_values = pd.read_csv('test_values.csv',index_col='building_id')
data_set = pd.read_csv('data_set.csv')

In [39]:
train, test = train_test_split(data_set, test_size = 0.20, shuffle = False)

In [40]:
train_values_subset = train.iloc[:,0:-1]
train_labels = train.loc[:, 'damage_grade']

In [41]:
test_values_subset = test.iloc[:,0:-1]
test_labels = test.loc[:, 'damage_grade']

In [42]:
# A parameter grid for XGBoost
n_jobs=[-1]
n_estimators=np.arange(100,1000,100)
learning_rate=[0.03,0.01,0.1]
max_depth=np.arange(10,100,15)
# Param grid for Xgboost
param_grid={'n_jobs':n_jobs,
            'n_estimators':n_estimators,
            'max_depth':max_depth,
            'learning_rate':learning_rate
           }

In [43]:
rf=XGBClassifier()
kf=KFold(n_splits=2,shuffle=True)

In [44]:
#rs=RandomizedSearchCV(rf,param_distributions=param_grid,cv=kf,scoring='f1_micro')

In [45]:
#random_search = RandomizedSearchCV(rf_model, param_distributions=params, n_iter=500, scoring='accuracy', n_jobs=1, cv=skf.split(train_values_subset,train_labels.values.ravel()), random_state=1)

In [46]:
train_values_subset = pd.get_dummies(train_values_subset)

In [47]:
# *----- Submit 0.7233
rf_model = XGBClassifier(n_jobs=-1,n_estimators= 600, max_depth= 10,learning_rate= 0.1)

In [ ]:
start_time = timer(None) # timing starts from this point for "start_time" variable
rf_model.fit(train_values_subset, train_labels.values.ravel())
timer(start_time) # timing ends here for "start_time" variable

[14:19:01] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
preds = rf_model.predict(test_values_subset)
f1_score(test_labels, preds, average='micro')

In [ ]:
#rs.best_params_

In [ ]:
for i in range(11):
    test_values.loc[:, [vector_names[i]]] = test_values.loc[:, [vector_names[i]]]*values[i]
test_values['sum_columns'] = 0
for x in range(11):
    test_values['sum_columns'] += test_values[vector_names[x]]

In [ ]:
foundation = pd.DataFrame()


land_surface = pd.DataFrame()
land_surface[['n', 'o', 't']]= pd.get_dummies(test_values['land_surface_condition'])
foundation[['h', 'i', 'r', 'u', 'w']]= pd.get_dummies(test_values['foundation_type'])
foundation['h'] = foundation['h'] * foundation['h'].mean()
foundation['i'] = foundation['i'] * foundation['i'].mean()
foundation['r'] = foundation['r'] * foundation['r'].mean()
foundation['u'] = foundation['u'] * foundation['u'].mean()
foundation['w'] = foundation['w'] * foundation['w'].mean()

land_surface['n']  = land_surface['n'] * land_surface['n'].mean()
land_surface['o']  = land_surface['o'] * land_surface['o'].mean()
land_surface['t']  = land_surface['t'] * land_surface['t'].mean()

test_values['base_condition'] = land_surface['n'] + land_surface['o'] + land_surface['t'] + foundation['h'] + foundation['i'] + foundation['r'] + foundation['u'] + foundation['w']

In [ ]:
test_values_subset = test_values[columnas_seleccionadas]
test_values_subset = pd.get_dummies(test_values_subset)

In [ ]:
predictions = rf_model.predict(test_values_subset)

In [ ]:
submission_format = pd.read_csv('submission_format.csv',index_col='building_id')

In [ ]:
my_submission = pd.DataFrame(data=predictions, columns=submission_format.columns,
                            index=submission_format.index)

In [ ]:
my_submission.head()

In [ ]:
my_submission.to_csv('submision.csv')

In [ ]:
!head submision.csv

In [ ]:
# Gini importance
rf_model.feature_importances_

In [ ]:
plt.bar(test_values_subset.columns, rf_model.feature_importances_)
plt.xlabel('Features')
plt.ylabel('Importancia')
plt.title('Importancia Features con RF')
plt.show()